In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from math import pi
import pandas as pd

def concat_images_vertically(img1, img2):
    width1, height1 = img1.size
    width2, height2 = img2.size
    new_img = Image.new('RGB', (max(width1, width2), height1 + height2))
    new_img.paste(img1, (0, 0))
    new_img.paste(img2, (0, height1))
    return new_img

def concat_images_horizontally(img_left, img_right):
    width_left, height_left = img_left.size
    width_right, height_right = img_right.size
    new_img = Image.new('RGB', (width_left + width_right, max(height_left, height_right)))
    new_img.paste(img_left, (0, 0))
    new_img.paste(img_right, (width_left, 0))
    return new_img

def draw_radar_chart(ax, data, labels, color):
    num_labels = len(labels)
    angles = [x / float(num_labels) * (2 * pi) for x in range(num_labels)]
    angles += angles[:1]
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    plt.xticks(angles[:-1], labels, fontsize=13)
    ax.tick_params(axis='x', which='major', pad=15)
    ax.set_rlabel_position(0)
    plt.yticks([0, 2, 4, 6, 8, 10], ['0', '2', '4', '6', '8', '10'], fontsize=10)
    plt.ylim(0, 10)
    data += data[:1]
    ax.plot(angles, data, color=color, linewidth=2, linestyle='solid')
    ax.fill(angles, data, color=color, alpha=0.4)

def draw_bar_chart(ax, data, labels, color):
    ax.bar(labels, data, color=color, alpha=0.7)
    ax.set_ylim(0, 10)
    plt.xticks(rotation=45, ha='right', fontsize=13)
    ax.set_yticks([0, 2, 4, 6, 8, 10])
    ax.set_yticklabels(['0', '2', '4', '6', '8', '10'], fontsize=10)

def process_image_folders(folder1, folder2, output_folder, img_right_path, radar_data, bar_data):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    images1 = sorted(os.listdir(folder1))
    images2 = sorted(os.listdir(folder2))

    img_right = Image.open(img_right_path)
    img_right = img_right.resize((1000, 2160))

    labels = radar_data.columns[1:]
    bar_labels = bar_data.columns[1:]
    my_palette = plt.cm.get_cmap("Set2", len(radar_data.index))

    for img1_name, img2_name, (_, radar_row), (_, bar_row) in zip(images1, images2, radar_data.iterrows(), bar_data.iterrows()):
        try:
            img1_path = os.path.join(folder1, img1_name)
            img2_path = os.path.join(folder2, img2_name)

            img1 = Image.open(img1_path)
            img2 = Image.open(img2_path)

            concatenated_img_vert = concat_images_vertically(img1, img2)

            fig = plt.figure(figsize=(8, 16))
            fig.set_facecolor('white')

            # Create radar chart
            ax_radar = fig.add_subplot(211, polar=True)
            color = my_palette(0)
            radar_data_values = radar_row.drop('Character').tolist()
            draw_radar_chart(ax_radar, radar_data_values, labels, color)

            # Create bar chart
            ax_bar = fig.add_subplot(212)
            bar_data_values = bar_row.drop('Character').tolist()
            draw_bar_chart(ax_bar, bar_data_values, bar_labels, color)

            fig.canvas.draw()
            combined_chart_image = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
            combined_chart_image = combined_chart_image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            combined_chart_image = Image.fromarray(combined_chart_image)
            combined_chart_image = combined_chart_image.resize((1000, 2160))
            plt.close(fig)

            combined_img_right = concat_images_horizontally(combined_chart_image, img_right)
            final_concatenated_img = concat_images_horizontally(concatenated_img_vert, combined_img_right)

            output_path = os.path.join(output_folder, f"concat_{img1_name}")
            final_concatenated_img.save(output_path)
        except Exception as e:
            print(f"Error processing {img1_name} and {img2_name}: {e}")

# Example usage
folder1 = 'C:/Users/jk/action_assess_3_chart/data/image/Z76/313-2-1-15-Z76_D'
folder2 = 'C:/Users/jk/action_assess_3_chart/data/image/Z106/313-2-1-15-Z106_D'
output_folder = 'C:/Users/jk/action_assess_3_chart/data/image/concat/313-2-1-15-Z106_D'
img_right_path = 'C:/Users/jk/action_assess_3_chart/pose_landmarks_index.png'

# Example radar data
radar_data = pd.DataFrame({
    'Character': ['A', 'A', 'A', 'B', 'B', 'B'],
    'Metric1': [5, 6, 7, 4, 5, 6],
    'Metric2': [6, 7, 8, 5, 6, 7],
    'Metric3': [8, 9, 7, 6, 5, 4],
    'Metric4': [7, 3, 5, 4, 3, 2],
    'Metric5': [6, 2, 4, 3, 2, 1],
    'Metric6': [5, 4, 3, 2, 1, 3]
})

# Example bar chart data
bar_data = pd.DataFrame({
    'Character': ['A', 'A', 'A', 'B', 'B', 'B'],
    'Metric1': [5, 5, 6, 7, 6, 5],
    'Metric2': [4, 5, 5, 6, 6, 7],
    'Metric3': [8, 7, 6, 5, 4, 5],
    'Metric4': [3, 2, 4, 5, 6, 4],
    'Metric5': [7, 6, 5, 4, 3, 2],
    'Metric6': [4, 5, 6, 5, 4, 3]
})

process_image_folders(folder1, folder2, output_folder, img_right_path, radar_data, bar_data)


C:\Users\HYVision\AppData\Local\Temp\ipykernel_24204\1587097255.py:58: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  my_palette = plt.cm.get_cmap("Set2", len(radar_data.index))
C:\Users\HYVision\AppData\Local\Temp\ipykernel_24204\1587097255.py:85: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  combined_chart_image = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\HYVision\AppData\Local\Temp\ipykernel_24204\1587097255.py:85: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  combined_chart_image = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\HYVision\AppDat